In [15]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q htt  https://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 256 kB in 2s 

In [55]:
df=spark.read.format("csv").option("header","true").option("inferschema","true").load("/content/mgo.csv");
df.show()

+-------+----+------+---------+
|Country| Age|Salary|Purchased|
+-------+----+------+---------+
| France|  44| 72000|       No|
|  Spain|  27| 48000|      Yes|
|Germany|  30| 54000|       No|
|  Spain|  38| 61000|       No|
|Germany|  40|  null|      Yes|
| France|  35| 58000|      Yes|
|  Spain|null| 52000|       No|
| France|  48| 79000|      Yes|
|Germany|  50| 83000|       No|
| France|  37| 67000|      Yes|
+-------+----+------+---------+



In [56]:
df.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Purchased: string (nullable = true)



In [68]:
(df.schema['Country'].dataType)

StringType

In [58]:
ms=df.columns;
gs=[];
for i in ms:
  gs.append(df.filter(df[i].isNull()).count());

In [59]:
gs

[0, 1, 1, 0]

In [60]:
from pyspark.sql.functions import *

In [61]:
ms=df.columns;
input_columns=[];
for i in ms:
  if str(df.schema[i].dataType) in ('IntegerType','DoubleType'):
    input_columns.append(i);

In [62]:
input_columns

['Age', 'Salary']

In [63]:
def fill_using_mean(dfa,cola):
  for c in dfa.columns:
    if c in cola:
      ms=dfa.select(mean(c).alias("Ammao")).collect()[0].asDict()['Ammao'];
      dfa=dfa.na.fill({c:ms});
  return dfa;

In [64]:
dfa=fill_using_mean(df,input_columns)

38.77777777777778
63777.77777777778


In [65]:
dfa.show()

+-------+---+------+---------+
|Country|Age|Salary|Purchased|
+-------+---+------+---------+
| France| 44| 72000|       No|
|  Spain| 27| 48000|      Yes|
|Germany| 30| 54000|       No|
|  Spain| 38| 61000|       No|
|Germany| 40| 63777|      Yes|
| France| 35| 58000|      Yes|
|  Spain| 38| 52000|       No|
| France| 48| 79000|      Yes|
|Germany| 50| 83000|       No|
| France| 37| 67000|      Yes|
+-------+---+------+---------+

